### Importing Libraries

In [1]:
import json
import pandas as pd

### Loading Data

Data Source : [layoffs.fyi](https://layoffs.fyi/)

In [2]:
with open('layoff_2023.txt') as f:
    json_data = json.load(f)

### Data Preprocessing

In [3]:
key_map = {}
            
for item in json_data['data']['table']['columns']:
    id_ = item['id']
    name_ = item['name']
    key_map[id_] = name_
    
#Location HQ
for item in json_data['data']['table']['columns']:
    if item['id']=='fld1mbRJlknicV1Dk':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_
    
#Industry
for item in json_data['data']['table']['columns']:
    if item['id']=='fldMvtAoxHCri39uZ':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

#Country
for item in json_data['data']['table']['columns']:
    if item['id']=='fldnR6AcR7HjRtwrG':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

#Stage
for item in json_data['data']['table']['columns']:
    if item['id']=='fldbWCltYdMzywX2v':
        for i in item['typeOptions']['choices'].values():
            id_ = i['id']
            name_ = i['name']
            key_map[id_] = name_

In [4]:
row_data = []
for item in json_data['data']['table']['rows']:
    item['cellValuesByColumnId']['id'] = item['id']
    row_data.append(item['cellValuesByColumnId'])

In [5]:
def replace_values(data, replacements):
    for item in data:
        for key, value in item.items():
            if key in ['fld1mbRJlknicV1Dk','fldMvtAoxHCri39uZ','fldbWCltYdMzywX2v','fldnR6AcR7HjRtwrG']:
                if isinstance(value, list):
                    item[key] = [replacements[item] for item in value]
                else:
                    item[key] = replacements[value]

replace_values(row_data, key_map)

In [6]:
def replace_keys(data, replacements):
    for item in data:
        item_copy = item.copy()
        for key, value in item_copy.items():
            item[replacements.get(key, key)] = item.pop(key, None)

replace_keys(row_data, key_map)

In [7]:
dataframe = pd.DataFrame(row_data)
dataframe['Date'] = pd.to_datetime(dataframe['Date']).dt.date
dataframe['Date Added'] = pd.to_datetime(dataframe['Date Added']).dt.date
dataframe.head()

,Company,Location HQ,Industry,%,Date,Source,$ Raised (mm),Stage,Date Added,Country,id,# Laid Off,List of Employees Laid Off
0,Buzzer,[New York City],Consumer,1.00,2023-06-30,https://frontofficesports.com/buzzer-the-gen-z...,24.0,Series A,2023-07-01,United States,rec0XEKZ07OHJDxQZ,NaN,NaN
1,Lunya,[Los Angeles],Retail,1.00,2023-06-30,https://www.modernretail.co/operations/dtc-sle...,NaN,Unknown,2023-07-01,United States,rec60rBOE1tRWAAQX,NaN,NaN
2,MediaMath,[New York City],Marketing,1.00,2023-06-30,https://www.adexchanger.com/online-advertising...,607.0,Private Equity,2023-07-01,United States,recm6MwYDpAfh6P5Z,NaN,NaN
3,Niantic,[SF Bay Area],Other,NaN,2023-06-29,https://www.cnbc.com/2023/06/29/pokemon-go-mak...,770.0,Series D,2023-06-29,United States,rec4joRZxafa63noN,230.0,NaN
4,Headspace,[Los Angeles],Healthcare,0.15,2023-06-29,https://www.latimes.com/business/story/2023-06...,216.0,Unknown,2023-06-30,United States,reczAqCzSqKoz1gic,181.0,NaN


In [8]:
from datetime import date

data_dict = dataframe.to_dict(orient='records')

data_dict_serializable = [{k: v.strftime('%Y-%m-%d') if isinstance(v, date) else v for k, v in row.items()} for row in data_dict]

with open('Layoffs - Preprocessed.json', 'w') as json_file:
    json.dump(data_dict_serializable, json_file)